## 1. Import Dependencies

In [15]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import re

## 2. Read post data into list

In [16]:
data = pd.read_csv('influencers_data.csv')

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('...see more', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = " ".join(filter(lambda x:x[0]!="@", text.split()))
    return text

corpus = data['content'].apply(lambda x: clean_text(x)).tolist()

## 3. Tokenization

In [17]:
tokenizer = Tokenizer(
    filters='"$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n'
)

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

## 4. Turn tokenized sentences into training data

In [18]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad Input Sequences
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

# Create predictors and labels
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

## 5. Train neural network

In [19]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
#print model.summary()
print(model)

/home/wingp/.local/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-02-28 04:18:33.961984: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 690113880 exceeds 10% of free system memory.


Epoch 1/100
879/879 [==============================] - 247s 277ms/step - loss: 7.1456 - accuracy: 0.0582
Epoch 2/100
879/879 [==============================] - 232s 264ms/step - loss: 6.2700 - accuracy: 0.0950
Epoch 3/100
879/879 [==============================] - 230s 262ms/step - loss: 5.7360 - accuracy: 0.1192
Epoch 4/100
879/879 [==============================] - 243s 276ms/step - loss: 4.6297 - accuracy: 0.1741
Epoch 5/100
879/879 [==============================] - 238s 271ms/step - loss: 3.7815 - accuracy: 0.2541
Epoch 6/100
879/879 [==============================] - 249s 283ms/step - loss: 3.2274 - accuracy: 0.3214
Epoch 7/100
879/879 [==============================] - 241s 275ms/step - loss: 2.8330 - accuracy: 0.3731
Epoch 8/100
879/879 [==============================] - 238s 271ms/step - loss: 2.5655 - accuracy: 0.4181
Epoch 9/100
879/879 [==============================] - 262s 298ms/step - loss: 2.3296 - accuracy: 0.4578
Epoch 10/100
879/879 [==============================] -

879/879 [==============================] - 235s 268ms/step - loss: 1.2412 - accuracy: 0.6755
Epoch 79/100
879/879 [==============================] - 277s 316ms/step - loss: 1.2781 - accuracy: 0.6653
Epoch 80/100
879/879 [==============================] - 257s 292ms/step - loss: 1.2473 - accuracy: 0.6743
Epoch 81/100
879/879 [==============================] - 295s 336ms/step - loss: 1.2143 - accuracy: 0.6787
Epoch 82/100
879/879 [==============================] - 304s 345ms/step - loss: 1.1725 - accuracy: 0.6862
Epoch 83/100
879/879 [==============================] - 338s 384ms/step - loss: 1.1786 - accuracy: 0.6879
Epoch 84/100
879/879 [==============================] - 276s 314ms/step - loss: 1.2106 - accuracy: 0.6815
Epoch 85/100
879/879 [==============================] - 252s 286ms/step - loss: 1.2461 - accuracy: 0.6744
Epoch 86/100
879/879 [==============================] - 298s 340ms/step - loss: 1.1894 - accuracy: 0.6861
Epoch 87/100
879/879 [==============================] - 278

In [21]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

2022-02-28 21:40:31.711548: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [26]:
seed_text = "We need to"
next_words = 50
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list),axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
    
print(seed_text)

We need to have ownership of the life i have been focusing on the process of my life with some of the coolest of problems. but do you prioritize #founders #scale #advantage both and agreed dates for some people or passionately want he who are late in the future ‘re shoot hundreds of
